## Dependencies

In [1]:
import re
import warnings
import pandas as pd
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, Average, BatchNormalization, Activation
from googleqa_utilityscript import *

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
model_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/use.h5'
category_encoder_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/category_encoder.joblib'
scaler_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/scaler.joblib'
tfidf_title_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tfidf_title.joblib'
tfidf_body_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tfidf_body.joblib'
tfidf_answer_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tfidf_answer.joblib'
tsvd_title_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tsvd_title.joblib'
tsvd_body_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tsvd_body.joblib'
tsvd_answer_path = '/kaggle/input/46-googleq-a-train-use-tf-idf-bigram/tsvd_answer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())
    
# Text length
test['question_title_len'] = test['question_title'].apply(lambda x : len(x))
test['question_body_len'] = test['question_body'].apply(lambda x : len(x))
test['answer_len'] = test['answer'].apply(lambda x : len(x))

# Word count
test['question_title_word_cnt'] = test['question_title'].apply(lambda x : len(x.split(' ')))
test['question_body_word_cnt'] = test['question_body'].apply(lambda x : len(x.split(' ')))
test['answer_word_cnt'] = test['answer'].apply(lambda x : len(x.split(' ')))

# Unique word count
test['question_title_unique_word_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' '))))
test['question_body_unique_word_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' '))))
test['answer_unique_word_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' '))))

# Misspelings count
misspelling_mapping_set = set(misspelling_mapping.keys())
test['question_title_misspelings_cnt'] = test['question_title'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['question_body_misspelings_cnt'] = test['question_body'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))
test['answer_misspelings_cnt'] = test['answer'].apply(lambda x : len(set(x.split(' ')) & misspelling_mapping_set))

## Test set

In [5]:
ohe_category = load(category_encoder_path)
scaler = load(scaler_path)
tfidf_title = load(tfidf_title_path)
tfidf_body = load(tfidf_body_path)
tfidf_answer = load(tfidf_answer_path)
tsvd_title = load(tsvd_title_path)
tsvd_body = load(tsvd_body_path)
tsvd_answer = load(tsvd_answer_path)


# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']
X_test_category = ohe_category.transform(test[['category']]).toarray()
X_test_dense = test[['question_title_len', 'question_title_word_cnt', 'question_title_unique_word_cnt', 'question_title_misspelings_cnt', 
                     'question_body_len', 'question_body_word_cnt', 'question_body_unique_word_cnt', 'question_body_misspelings_cnt', 
                     'answer_len', 'answer_word_cnt', 'answer_unique_word_cnt', 'answer_misspelings_cnt']]

X_test_title_tfidf = tfidf_title.transform(test["question_title"].values)
X_test_body_tfidf = tfidf_body.transform(test["question_body"].values)
X_test_answer_tfidf = tfidf_answer.transform(test["answer"].values)
X_test_title_tfidf = tsvd_title.transform(X_test_title_tfidf)
X_test_body_tfidf = tsvd_body.transform(X_test_body_tfidf)
X_test_answer_tfidf = tsvd_answer.transform(X_test_answer_tfidf)

X_test_dense = scaler.transform(X_test_dense)
X_test = [X_test_title, X_test_body, X_test_answer, X_test_category, X_test_dense, X_test_title_tfidf, X_test_body_tfidf, X_test_answer_tfidf]

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)

# Model

In [7]:
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

def encoder_block(input_layer):
    encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
    return encoder

In [8]:
# Title
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
encoded_title = encoder_block(input_title)

# Body
input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
encoded_body = encoder_block(input_body)

# Answer
input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
encoded_answer = encoder_block(input_answer)

# Category
input_category = Input(shape=(X_test_category.shape[1],), dtype=tf.float32, name='input_category')

# Dense features
input_dense = Input(shape=(X_test_dense.shape[1],), dtype=tf.float32, name='input_dense')

# Title TF-IDF
input_title_tfidf = Input(shape=(X_test_title_tfidf.shape[1],), dtype=tf.float32, name='input_title_tfidf')

# Body TF-IDF
input_body_tfidf = Input(shape=(X_test_body_tfidf.shape[1],), dtype=tf.float32, name='input_body_tfidf')

# Answer TF-IDF
input_answer_tfidf = Input(shape=(X_test_answer_tfidf.shape[1],), dtype=tf.float32, name='input_answer_tfidf')


# Output
dense = Concatenate()([input_dense, input_category])
dense = Dense(32, kernel_initializer='glorot_uniform')(dense)
dense = BatchNormalization()(dense)
dense = Activation('relu')(dense)
dense = Dropout(0.2)(dense)
dense = Dense(16, kernel_initializer='glorot_uniform')(dense)
dense = BatchNormalization()(dense)
dense = Activation('relu')(dense)
dense = Dropout(0.2)(dense)

title = Concatenate()([encoded_title, input_title_tfidf])
title = Dense(128, kernel_initializer='glorot_uniform')(title)
title = BatchNormalization()(title)
title = Activation('relu')(title)
title = Dropout(0.2)(title)

body = Concatenate()([encoded_body, input_body_tfidf])
body = Dense(128, kernel_initializer='glorot_uniform')(body)
body = BatchNormalization()(body)
body = Activation('relu')(body)
body = Dropout(0.2)(body)

answer = Concatenate()([encoded_answer, input_answer_tfidf])
answer = Dense(128, kernel_initializer='glorot_uniform')(answer)
answer = BatchNormalization()(answer)
answer = Activation('relu')(answer)
answer = Dropout(0.2)(answer)

question = Concatenate()([title, body])
question = Dense(128, kernel_initializer='glorot_uniform')(question)
question = BatchNormalization()(question)
question = Activation('relu')(question)
question = Dropout(0.2)(question)

x = Concatenate()([question, answer, dense, encoded_title, encoded_body, encoded_answer])
x = Dense(512, kernel_initializer='glorot_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='glorot_uniform', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer, input_category, input_dense, input_title_tfidf, input_body_tfidf, input_answer_tfidf], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
__________________________________________________________________________________________________
input_title_tfidf (InputLayer)  [(None, 100)]        0                                            
______________________________________________________________________________________________

# Make predictions

In [9]:
Y_test = model.predict(X_test)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.805836,0.698705,0.095262,0.752456,0.784735,0.666975,0.636467,0.573539,0.254318,...,0.884855,0.863076,0.567447,0.894236,0.935548,0.722603,0.067934,0.078051,0.710486,0.865166
1,46,0.810490,0.480218,0.002943,0.618016,0.897018,0.914353,0.556191,0.409261,0.304735,...,0.676831,0.904553,0.577823,0.963260,0.963837,0.786805,0.912755,0.086591,0.057921,0.815827
2,70,0.843389,0.542668,0.039237,0.817507,0.895275,0.945740,0.584940,0.499340,0.208644,...,0.782085,0.892570,0.598535,0.963229,0.957094,0.769140,0.090612,0.074777,0.696873,0.886016
3,132,0.819266,0.520953,0.011490,0.583925,0.781382,0.860692,0.561368,0.456933,0.289820,...,0.714227,0.888748,0.658669,0.951211,0.957286,0.833289,0.624309,0.175111,0.673026,0.858957
4,200,0.956745,0.551689,0.024195,0.890156,0.653126,0.890816,0.631279,0.651279,0.283550,...,0.759830,0.975170,0.705874,0.995047,0.988184,0.917316,0.246916,0.157551,0.786363,0.936325
